In [1]:
import pickle
import gradio as gr
import pandas as pd
# import numpy as np

c:\Users\kings\miniconda3\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import catboost
print(catboost.__version__)

1.2


In [15]:
def predict_car(brand, model, reg_year, mileage, condition, fuel, body, colour):
    # Prepare the input features for prediction
    input_features = pd.DataFrame({
        'mileage': [mileage],
        'standard_colour':[colour],
        'standard_make': [brand],
        'standard_model': [model],
        'vehicle_condition': [condition],
        'year_of_registration': [int(reg_year)],
        'body_type': [body],
        'fuel_type': [fuel],
    })

    enc = pickle.load(open("../car_price_app/encoder.pkl","rb"))
    input_features = enc.transform(input_features)
    model = pickle.load(open("../car_price_app/carmodel.pickle","rb"))

    # Make predictions on the input features
    predicted_price = model.predict(input_features)

    return "£{}".format(round(predicted_price[0], 2))


In [16]:
predict_car("VAUXHALL", "CROSSLANDX", 2020, 5984, "USED", "PETROL","SUV","WHITE")

14741.27

In [31]:
df = pd.read_csv("../data/sampled.csv")
df

,mileage,reg_code,standard_colour,standard_make,standard_model,vehicle_condition,year_of_registration,price,body_type,crossover_car_and_van,fuel_type,advert_year,vehicle_age,avg_miles_per_year
0,6000,69,WHITE,KIA,CEED,USED,2019,14495,HATCHBACK,0,DIESEL,2020,1,6000
1,60000,64,BLACK,AUDI,A1,USED,2014,8500,HATCHBACK,0,PETROL,2020,6,10000
2,14598,66,GREEN,KIA,SPORTAGE,USED,2016,16500,SUV,0,DIESEL,2020,4,3649
3,5267,20,BLUE,SKODA,SUPERB,USED,2020,29990,HATCHBACK,0,PETROLPLUG-INHYBRID,2020,0,5267
4,70928,16,BLACK,LANDROVER,DISCOVERYSPORT,USED,2016,18000,SUV,0,DIESEL,2020,4,17732
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36532,7605,69,GREY,MERCEDES-BENZ,CCLASS,USED,2019,24795,ESTATE,0,PETROL,2020,1,7605
36533,24000,63,YELLOW,FIAT,500C,USED,2013,6047,CONVERTIBLE,0,PETROL,2020,7,3428
36534,9800,65,BLUE,AUDI,S3,USED,2015,17995,HATCHBACK,0,PETROL,2020,5,1960
36535,96700,62,SILVER,KIA,OPTIMA,USED,2012,5274,SALOON,0,DIESEL,2020,8,12087


In [18]:
df['year_of_registration'].min()

1963

In [19]:
def search_car_model(search_term):
	result_df = df[df['standard_make'].str.contains(search_term)]
	return result_df['standard_model'].drop_duplicates().tolist()

In [20]:
def search_body(model):
    result_df = df[df['standard_model']==model]
    return result_df['body_type'].drop_duplicates().tolist()

In [21]:
search_body("CEED")

['HATCHBACK', 'ESTATE']

In [22]:
kia = df.query("standard_make=='KIA'")
kia.query("standard_model=='CEED'")['body_type'].unique()

array(['HATCHBACK', 'ESTATE'], dtype=object)

In [23]:
def search_fuel(model):
    result_df = df[df['standard_model']==model]
    return result_df['fuel_type'].drop_duplicates().tolist()

In [24]:
search_fuel("CEED")

['DIESEL', 'PETROL', 'PETROLPLUG-INHYBRID']

In [28]:
kia = df.query("standard_make=='KIA'")
ceed = kia.query("standard_model=='SPORTAGE'")
# ceed[ceed['fuel_type']=="DIESEL"]['year_of_registration'].unique()
ceed['fuel_type'].value_counts()

DIESEL    272
PETROL    152
Name: fuel_type, dtype: int64

In [29]:
fueltype = ['BIFUEL','DIESEL','DIESELHYBRID','DIESELPLUG-INHYBRID','ELECTRIC','PETROL','PETROLHYBRID','PETROLPLUG-INHYBRID']
bodytype = ['CAMPER','CONVERTIBLE','COUPE','ESTATE','HATCHBACK','LIMOUSINE','MINIBUS','MPV','PICKUP','SALOON','SUV','VANS']
colours = ['BEIGE','BLACK','BLUE','BRONZE','BROWN','GREEN','GREY','MULTICOLOUR','ORANGE','PURPLE','RED','SILVER','WHITE','YELLOW','OTHER COLOUR']
brands = df['standard_make'].drop_duplicates().values.tolist()
models = df['standard_model'].drop_duplicates().values.tolist()

In [30]:
brand_models = {brand: search_car_model(brand) for brand in brands}

In [16]:
model_body = {model: search_body(model) for model in models}

In [17]:
with gr.Blocks(gr.themes.Soft()) as demo:
    gr.Markdown(
    """
    # CAR PRICE PREDICTION

    ### QUICK GUIDE
    To enhance the user experience, Some engineering stunts were performed:
    1. When you choose a car brand, the available models for that brand will be updated automatically in the model dropdown menu. 
    2. Similarly, when you choose a car model, the available body types and fuel types will be updated in the body types and fuel types dropdown menu respectively.
    <br />
    <br />
    **NOTE**: the visibility of the car details section is updated when a brand is selected. This means that when you choose a brand, the additional car details section will become visible, year of registration, mileage, condition of the car(new used), and  specific details about the car. 
    
    """
    )
    colour = ['BEIGE','BLACK','BLUE','BRONZE','BROWN','GREEN','GREY','MULTICOLOUR','ORANGE','PURPLE','RED','SILVER','WHITE','YELLOW','OTHER COLOUR']
    brands = df['standard_make'].drop_duplicates().values.tolist()
    models = df['standard_model'].drop_duplicates().values.tolist()
    car_brand = gr.Dropdown(label="Brand", choices=brands)
    car_model = gr.Dropdown(label="Model", choices=[])
    body = gr.Dropdown(label='Body Type', choices=[])
    fuel = gr.Dropdown(label='Fuel Type', choices=[])

    with gr.Column(visible=False) as details_col:
        year = gr.Slider(1990, 2020, step=1, label="Year of Registration")
        mileage = gr.Textbox(label="Mileage", placeholder="Input mileage...")
        condition = gr.Radio(["New", "Used"], label="Vehicle Condition")
        colour = gr.Dropdown(colour, label="Colour")
        generate_btn = gr.Button("Predict Price")
        gr.Markdown("Predicted Price:")
        output = gr.Text(label="Predict Price")

    brand_models = {brand: search_car_model(brand) for brand in brands}
    model_body = {model: search_body(model) for model in models}
    model_fuel = {model: search_fuel(model) for model in models}


    def filter_models(car_brand):
        return gr.Dropdown.update(
            choices=brand_models[car_brand],value=brand_models[car_brand][0]
        ), gr.update(visible=True)

    car_brand.change(filter_models, car_brand,[car_model, details_col])

    def filter_body(car_model):
        return gr.Dropdown.update(
            choices=model_body[car_model],value=model_body[car_model][0]
        ), gr.update(visible=True)

    car_model.change(filter_body, car_model,[body, details_col])

    def filter_fuel(car_model):
        return gr.Dropdown.update(
            choices=model_fuel[car_model],value=model_fuel[car_model][0]
        ), gr.update(visible=True)

    car_model.change(filter_fuel, car_model,[fuel, details_col])

    generate_btn.click(fn=predict_car, inputs=[car_brand, car_model, year, mileage, condition, fuel, body, colour],outputs=output)


# if __name__ == "__main__":
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
